In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [2]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
import gc

from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, Add, GRU

np.random.seed(42)
tf.random.set_seed(42)

In [4]:
df = pd.read_csv('/kaggle/input/qqqqqqqq/Traffic_Data.csv')
print(f"train_df: {df.shape}")
df.head()


In [5]:
df.columns

In [6]:
df['time'] = range(1, len(df) + 1)


In [7]:
df.head()

In [8]:
def Time_Series_Features(df):
    df['cross1']= df['car'] * df['truck']
    df['cross2']= df['car'] * df['bus']
    df['cross3']= df['truck'] * df['bus']
   
    df.plot()
    print("Step-1...Completed")
    
    df['car_lag1'] = df['car'].shift(1)
    df['car_lag2'] = df['car'].shift(-1)
    df['truck_blag1'] = df['truck'].shift(1)
    df['truck_blag2'] = df['truck'].shift(-1)
    
    df['bus_lag1'] = df['bus'].shift(-1)
    df['bus_lag2'] = df['bus'].shift(1)
    
    df = df.fillna(0)
    print("Step-2...Completed")
    
    df['car__max'] = df['car'].max()
    df['truck__max'] = df['truck'].max()
    df['truck__diffmax'] = df['truck'].max() - df['truck']
    df['car__diffmean'] = df['car'].mean() - df['car']
    df['bus__diffmax'] = df['bus'].max() - df['bus']
    df['car__diffmean'] = df['car'].mean() - df['car']
    print("Step-3...Completed")
    
    df['car_diff1'] = df['car'] - df['car_lag1']
    df['car_diff2'] = df['car'] - df['car_lag2']
    df['bus_diff1'] = df['bus'] - df['bus_lag1']
    df['bus_diff2'] = df['bus'] - df['bus_lag2']
    
    print("Step-4...Completed")
    
    
    
    return df

In [59]:
df.plot(x="time",y=["car","truck","bus"],kind='line')

In [60]:
df.plot(x="time",y=["cross1","cross2","cross3"],kind='line')
    

In [ ]:
df.plot(x="time",y=["car","truck","bus"],kind='bar')

In [41]:
plt.bar(df["time"],df["truck"])
plt.xlabel("Time")
plt.ylabel("No of truck")
plt.legend()
plt.savefig("truck.png",dpi=600)
plt.show()

In [42]:
plt.bar(df["time"],df["bus"])
plt.xlabel("Time")
plt.ylabel("No of Bus")
plt.legend()
plt.savefig("Bus.png",dpi=600)
plt.show()

In [46]:
df.plot()

In [11]:
df.describe()

In [47]:
train=Time_Series_Features(df)

In [13]:
train.columns

In [14]:
train=train.drop(['frame','time'],axis=1)

In [15]:
T=np.array(train)
T.shape

In [16]:
nput=[]
Target=[]

In [17]:
for i in range(1650):
    nput.append(T[0+i:30+i,:])
    Target.append(T[30+i:60+i,:])

In [18]:
nput=np.array(nput)

Target=np.array(Target)
Target.shape,nput.shape

In [19]:
K=nput.shape
print(K[1:])

In [20]:
z=32

In [30]:
def get_model():
    enc_in = Input(shape=(K[1:]))
    x1 = Bidirectional(LSTM(units=128, return_sequences=True))(enc_in)
    x2 = Bidirectional(LSTM(units=64, return_sequences=True))(x1)
    x3 = Bidirectional(LSTM(units=32, return_sequences=True))(x2)
    z2 = Bidirectional(GRU(units=128, return_sequences=True))(x2)
    Ex = Concatenate(axis=2)([x3, z2])
    
    EX = Dense(units=32, activation='selu')(Ex)
    
    

    encoder = tf.keras.Model(enc_in, EX)

    dec_in = Input(shape = (z,1))
    x6 = Bidirectional(LSTM(units=128, return_sequences=True))(EX)
    x7 = Bidirectional(LSTM(units=64, return_sequences=True))(x6)
    x8 = Bidirectional(LSTM(units=32, return_sequences=True))(x7)
    z6 = Bidirectional(GRU(units=128, return_sequences=True))(x8)
    Dx = Concatenate(axis=2)([x8, z6])
    
    DX = Dense(units=32, activation='selu')(Dx)

    dec_out=Dense(units=24)(DX)
    #decoder = tf.keras.Model(dec_in, dec_out)
   

    #dec_out = decoder(encoder(enc_in)[2])

    ae = tf.keras.Model(enc_in, dec_out)

    return {'inputs': enc_in, 
            'outputs': dec_out,
            'encoder': encoder,
            #'decoder': decoder,
            'ae': ae}

In [31]:
model = get_model()
ae=model['ae']
ae.summary()

In [32]:
plot_model(
    ae, 
    to_file='ae.png', 
    show_shapes=True,
    show_layer_names=True
)

In [33]:
ae.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
history=ae.fit(
    x=nput, y=Target, epochs=150,batch_size=100, validation_split=0.3
)

In [39]:
import matplotlib.pyplot as plt

plt.plot(history.history["val_loss"],label='validation_loss')

plt.plot(history.history["loss"],label='Training_loss')
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.legend()
plt.savefig("Train_valid_loss_mse.png",dpi=600)
plt.show()

In [ ]:
ae.save('ae1')

In [38]:
plt.plot(history.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Valid Losses")
plt.legend()
plt.savefig("valid_loss_mse.png",dpi=600)
plt.show()
plt.plot(history.history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Train Losses")
plt.legend()
plt.savefig("Train_loss_mse.png",dpi=600)
plt.show()